In [2]:
import torch
import pickle
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model_names = [
    'model_51_3_PL.pkl',
    'model_51_4_FL.pkl',
    'model_429_3_FL.pkl',
    'model_52_4_PL.pkl',
]
models = []
for m in model_names:
    with open(m, 'rb') as file:
        cur_model = pickle.load(file)        
        cur_model = cur_model.to(torch.float32)
        models.append(cur_model)

In [4]:
with open('test_set.pkl', 'rb') as file:
    test_set = pickle.load(file)

In [5]:
def test_model(model):
    test_losses = []
    y_preds = []
    simple_loss = nn.MSELoss(reduction='mean')
    for batch_graph, graph_labels in test_set:
        if torch.cuda.is_available():
            batch_graph = batch_graph.to(torch.cuda.current_device())
            graph_labels = graph_labels.cuda()
        model.eval()
        with torch.no_grad():
            batch_graph.ndata['feat'] = batch_graph.ndata['feat'].float()
            ypred = model(batch_graph)
            y_preds.append(ypred)
            loss = simple_loss(ypred, torch.tensor([graph_labels], dtype=torch.float32))
            if loss.item() < 50 or torch.any(ypred > 50): # Throw out outliers
                test_losses.append(loss.item())
    return test_losses, y_preds

In [ ]:
losses, preds = [], []
for i in range(4):
    l, p = test_model(models[i])
    losses.append(np.mean(l))
    preds.append(p)

In [ ]:
[np.mean(x) for x in losses]

## Standard Deviation

In [ ]:
[np.array(x).std() for x in preds]

In [ ]:
np.array([x[1] for x in test_set if x[1] < 50]).std()